In [51]:
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts 
from statsmodels.tsa.stattools import coint
import numpy as np
import pandas as pd

In [44]:
data = pd.read_csv('datasets/data.csv')

In [52]:
def find_cointegrated_pairs(data):
    n = data.shape[1]
    score_matrix = np.zeros((n, n))
    pvalue_matrix = np.ones((n, n))
    keys = data.keys()
    pairs = []
    for i in range(n):
        for j in range(i+1, n):
            y = data[keys[i]]
            x = data[keys[j]]
            result = coint(y, x)
            score = result[0]
            pvalue = result[1]
            
            if pvalue < 0.02:
                score_matrix[i, j] = score
                pvalue_matrix[i, j] = pvalue
                pairs.append((keys[i], keys[j]))

    return score_matrix, pvalue_matrix, pairs

In [53]:
find = data[data.columns.difference(['Data', 'Unnamed: 0'])]
score_matrix, pvalue_matrix, pairs = find_cointegrated_pairs(find)

(array([[ 0.        , -1.08322414, -3.04113271, ..., -2.37331647,
         -4.77004192, -2.38853967],
        [ 0.        ,  0.        , -0.71958123, ..., -0.66762336,
         -0.77178175, -0.68352743],
        [ 0.        ,  0.        ,  0.        , ..., -2.54415519,
         -2.92878247, -1.92706779],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         -2.32915888, -2.59079459],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        , -2.60240271],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]]),
 array([[1.00000000e+00, 8.85892286e-01, 1.00748955e-01, ...,
         3.37594805e-01, 4.26042304e-04, 3.30301918e-01],
        [1.00000000e+00, 1.00000000e+00, 9.44483817e-01, ...,
         9.50020224e-01, 9.38312382e-01, 9.48386784e-01],
        [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
         2.59950064e-01, 1.28119075e-01, 5.66218049e-01],
        ...,


In [17]:
data.columns.difference(['data'])

Index(['AALR3', 'AAPL34', 'ABCB4', 'ABEV3', 'ADHM3', 'AGRO3', 'ALPA3', 'ALPA4',
       'ALSO3', 'ALUP11',
       ...
       'VLID3', 'VULC3', 'VVAR3', 'WEGE3', 'WHRL4', 'WIZS3', 'WSON33',
       'XBOV11', 'XPIE11', 'YDUQ3'],
      dtype='object', length=268)